In [1]:
import cv2
import numpy as np
import pandas as pd
import os
import sys
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
import keras_tuner as kt
from sklearn.metrics import classification_report
import shutil
from os import path
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing import image_dataset_from_directory
from keras.preprocessing.image import ImageDataGenerator

# path variables to access files
root = os.getcwd()

### Load dataframe

In [2]:
main_data = pd.read_csv("data_labels_mainData.csv")
extra_data = pd.read_csv("data_labels_extraData.csv")

### Get data from splitted folder

In [3]:
# method get data for multiclass task (task 2)
def celltype_classify_data(class_list):    
    images = list()
    labels = list()
    
    for i, label in enumerate(class_list):
        # get image directory
        img_dir = os.path.join(root, "multiclass_task", f"{label}")
        
        for img in os.listdir(img_dir):
            img = cv2.imread(os.path.join(img_dir, img))
            # resize to 0-1 for faster computation
            resized = img / 255
            images.append(resized)
            labels.append(i)
        
    return (images, labels)

### Classification report method

In [4]:
def multiclass_classification_report(y_test, prediction):
    """
        Method to generate sklearn classification report with CNN multiclass output
    """
    
    encoded_pred = list()
    # convert each CNN output (sparse categorial) to class
    for pred in prediction:
        encoded_pred.append(np.argmax(pred))

    encoded_pred = np.array(encoded_pred)
    print(classification_report(y_test, encoded_pred))

### Structure of sub-models

In [7]:
def get_multiclass_model():
    """
        Structure of model classifying 4 classes
    """
    
    model = tf.keras.Sequential()
    # First convo-pooling
    # Convolutional layers (filter the image with a kernel)
    model.add(tf.keras.layers.Conv2D(64, (3, 3), strides=1,activation="relu", input_shape=[27, 27, 3]))
    # Max-pooling layers (reduce the size of the image by choosing max pixel at certain area)
    model.add(tf.keras.layers.MaxPool2D(pool_size=(2, 2), strides=1))
    
    # Second convo-pooling
    # Convolutional layers (filter the image with a kernel)
    model.add(tf.keras.layers.Conv2D(64, (3, 3),strides=1, activation="relu"))
    # Max-pooling layers (reduce the size of the image by choosing max pixel at certain area)
    model.add(tf.keras.layers.MaxPool2D(pool_size=(2, 2),strides=1))
    
    # Flatten input
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(2, activation="relu"))
    model.add(tf.keras.layers.Dense(128, activation="relu"))
    model.add(tf.keras.layers.Dense(32, activation="relu"))
    # Output layer
    model.add(tf.keras.layers.Dense(4, activation="softmax"))
    # Compile model 
    model.compile(optimizer="adam",
                  loss="sparse_categorical_crossentropy",
                  metrics=["accuracy"])
    
    return model

def get_subclass_model():
    """
        Structure of model classifying 3 classes
    """
    
    model = tf.keras.Sequential()
    # First convo-pooling
    # Convolutional layers (filter the image with a kernel)
    model.add(tf.keras.layers.Conv2D(64, (3, 3), activation="relu", input_shape=[27, 27, 3]))
    # Max-pooling layers (reduce the size of the image by choosing max pixel at certain area)
    model.add(tf.keras.layers.MaxPool2D(pool_size=(2, 2)))
    # Flatten input
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(256, activation="relu"))
    model.add(tf.keras.layers.Dense(128, activation="relu"))
    model.add(tf.keras.layers.Dense(32, activation="relu"))
    # Output layer
    model.add(tf.keras.layers.Dense(3, activation="softmax"))
    # Compile model 
    model.compile(optimizer="adam",
                  loss="sparse_categorical_crossentropy",
                  metrics=["accuracy"])
    
    return model

### Methods to train and save submodels

In [24]:
submodel_path = os.path.join(root, "multiclass_submodels")   
def three_class_submodel(file_path, aug_train = None, aug_val = None):
    # path to model trained with 3 classes
    subclass_path = path.join(file_path, "subclass.h5")
    subclass_model = None
    if not os.path.isfile(subclass_path) or keras.models.load_model(subclass_path) == None:
        print("---Sub model training---")
        # train subclass that fit with 3 types of images
        subclass_model = get_subclass_model()
        
        if aug_train is not None and aug_val is not None:
            print("Train with augmented data")
            subclass_model.fit(aug_train, epochs=40, validation_data=aug_val)
        else:
            print("Train with non-augmented data")
            subclass_model.fit(subx_train, suby_train, epochs=40, validation_data=(subx_test, suby_test))
            
        subclass_model.save(subclass_path)
        return subclass_model
    else:
        subclass_model = keras.models.load_model(subclass_path)
        print("Model trained with 3 classes loaded")
        return subclass_model
    
def all_class_submodel(file_path, aug_train = None, aug_val = None):
    allclass_path = path.join(file_path, "allclass.h5")
    allclass_model = None
    if not os.path.isfile(allclass_path) or keras.models.load_model(allclass_path) == None:
        print("---Sub model training---")
        # train subclass that fit with 4 types of images
        allclass_model = get_multiclass_model()
        
        if aug_train is not None and aug_val is not None:
            print("Train with augmented data")
            allclass_model.fit(aug_train, epochs=15, validation_data=aug_val)
        else:
            print("Train with non-augmented data")
            allclass_model.fit(x_train, y_train, epochs=15, validation_data=(x_val, y_val))
            
        allclass_model.save(allclass_path)
        return allclass_model
    else:
        allclass_model = keras.models.load_model(allclass_path)
        print("Model trained with 4 classes loaded")
        return allclass_model

### Meta-learner

In [6]:
def get_transfer_model():
    model = tf.keras.Sequential()
    # Flatten input
    model.add(tf.keras.layers.Flatten())
    # Hidden layers
    model.add(tf.keras.layers.Dense(256, activation="relu"))
    model.add(tf.keras.layers.Dense(128, activation="softmax"))
    model.add(tf.keras.layers.Dense(32, activation="relu"))
    model.add(tf.keras.layers.Dense(3, activation="sigmoid"))
    # Output layer
    model.add(tf.keras.layers.Dense(4, activation="softmax"))
    # Compile model
    model.compile(optimizer="adam",
                  loss="sparse_categorical_crossentropy",
                  metrics=["accuracy"])
    return model

### Get augmentated data for training

In [25]:
# load images from folders
all_class = ["epithelial", "fibroblast", "inflammatory", "others"]
cell_img, cell_label = celltype_classify_data(all_class)
cell_img, cell_label = np.array(cell_img), np.array(cell_label)

sub_multiclass = ["epithelial", "fibroblast", "inflammatory"]
sub_img, sub_label = celltype_classify_data(sub_multiclass)

# train and validate data for 3 classes
subx_train, subx_test, suby_train, suby_test = train_test_split(
    np.array(sub_img), np.array(sub_label), test_size=0.2
)

# train and validate data for 4 classes
x_train_val, x_test, y_train_val, y_test = train_test_split(
    cell_img, cell_label, test_size=0.2
)
x_train, x_val, y_train, y_val = train_test_split(
    x_train_val, y_train_val, test_size=0.25
)

# get augmentation for extra training data
extra_datagen = ImageDataGenerator(
    width_shift_range=0.3,
    height_shift_range=0.3,
    rotation_range=60,
    horizontal_flip=True,
    vertical_flip=True,

)

# fit augmentation with all images from all 4 classes
extra_datagen.fit(cell_img)

# get augmentations for unlabeled data
shift_aug = ImageDataGenerator(
    width_shift_range=0.3,
    height_shift_range=0.3,
)
rotation_aug = ImageDataGenerator(
    rotation_range=60
)
flip_aug = ImageDataGenerator(
    horizontal_flip=True,
    vertical_flip=True,
)
aug_list = [shift_aug, rotation_aug, flip_aug]

### Helper function 

In [26]:
def mix_match_pred(x, model):
#     get prediction of mix match model
    pred_arr = None
    
#     Find the sum of all predictions from each augmentation test set
    for aug in aug_list:
        test_gen = aug.flow(x)
        pred = model.predict_generator(test_gen, steps=len(x))
        if pred_arr is None:
            pred_arr = pred
        else:
            pred_arr = np.add(pred_arr, pred)
#     Find the average of prediction      
    pred_arr = np.true_divide(pred_arr, 3)
    return pred_arr

def get_extra_stacked(x, submodels, datagen):
    """Generate stacked output from submodels with augmented x for meta learner"""
    stacked = None
    print(len(x))
    # augmentate x before predicting with submodels
    pred_gen = datagen.flow(x)
    for submodel in submodels:
        if stacked is None:
            stacked = submodel.predict_generator(pred_gen)
        else:
            # augmentate x before predicting with submodels
            new_pred = submodel.predict_generator(pred_gen)
            stacked = np.concatenate((stacked, new_pred), axis=1)
    return stacked

In [27]:
extramodel_path = os.path.join(root, "extra_submodels")
# data for 3 classes submodels
sub_train = extra_datagen.flow(subx_train, suby_train)
sub_val = extra_datagen.flow(subx_test, suby_test)

# data for 4 classes submodels 
all_train = extra_datagen.flow(x_train, y_train)
all_val = extra_datagen.flow(x_val, y_val)

# train submodels with augmented x
subclass_extra = three_class_submodel(extramodel_path, aug_train=sub_train, aug_val=sub_val)
allclass_extra = all_class_submodel(extramodel_path, aug_train=all_train, aug_val=all_val)
extra_submodels = [subclass_extra, allclass_extra]

# get stacked x for meta-learner
stacked_x_train = get_extra_stacked(x_train,extra_submodels, extra_datagen)
stacked_x_val = get_extra_stacked(x_val,extra_submodels, extra_datagen)

print(stacked_x_train.shape)
print(stacked_x_val.shape)
print(y_train.shape)
print(y_val.shape)

extra_model = get_transfer_model()
# fit model with stacked data from submodels
extra_model.fit(stacked_x_train, y_train, epochs=15, validation_data=(stacked_x_val, y_val))

---Sub model training---
Train with augmented data
Epoch 1/40


2022-05-10 12:15:20.145172: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


207/213 [============================>.] - ETA: 0s - loss: 0.8419 - accuracy: 0.6060

2022-05-10 12:15:22.158078: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


213/213 [==============================] - 2s 10ms/step - loss: 0.8383 - accuracy: 0.6080 - val_loss: 0.6444 - val_accuracy: 0.7162
Epoch 2/40
213/213 [==============================] - 2s 9ms/step - loss: 0.6859 - accuracy: 0.6998 - val_loss: 0.5791 - val_accuracy: 0.7532
Epoch 3/40
213/213 [==============================] - 2s 9ms/step - loss: 0.6521 - accuracy: 0.7259 - val_loss: 0.6924 - val_accuracy: 0.7174
Epoch 4/40
213/213 [==============================] - 2s 9ms/step - loss: 0.6208 - accuracy: 0.7419 - val_loss: 0.5815 - val_accuracy: 0.7485
Epoch 5/40
213/213 [==============================] - 2s 9ms/step - loss: 0.6052 - accuracy: 0.7515 - val_loss: 0.5819 - val_accuracy: 0.7456
Epoch 6/40
213/213 [==============================] - 2s 9ms/step - loss: 0.5885 - accuracy: 0.7601 - val_loss: 0.5419 - val_accuracy: 0.7726
Epoch 7/40
213/213 [==============================] - 2s 9ms/step - loss: 0.5768 - accuracy: 0.7666 - val_loss: 0.5330 - val_accuracy: 0.7891
Epoch 8/40
213/2

...

1979
(5937, 7)
(1979, 7)
(5937,)
(1979,)
Epoch 1/15
  8/186 [>.............................] - ETA: 1s - loss: 1.3540 - accuracy: 0.4258 

2022-05-10 12:17:17.443392: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


186/186 [==============================] - ETA: 0s - loss: 1.3268 - accuracy: 0.4122

2022-05-10 12:17:18.728454: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


186/186 [==============================] - 2s 8ms/step - loss: 1.3268 - accuracy: 0.4122 - val_loss: 1.3032 - val_accuracy: 0.4164
Epoch 2/15
186/186 [==============================] - 1s 7ms/step - loss: 1.3077 - accuracy: 0.4122 - val_loss: 1.3008 - val_accuracy: 0.4164
Epoch 3/15
186/186 [==============================] - 1s 7ms/step - loss: 1.3071 - accuracy: 0.4122 - val_loss: 1.3008 - val_accuracy: 0.4164
Epoch 4/15
186/186 [==============================] - 1s 7ms/step - loss: 1.3072 - accuracy: 0.4122 - val_loss: 1.3004 - val_accuracy: 0.4164
Epoch 5/15
186/186 [==============================] - 1s 7ms/step - loss: 1.3071 - accuracy: 0.4122 - val_loss: 1.3007 - val_accuracy: 0.4164
Epoch 6/15
186/186 [==============================] - 1s 7ms/step - loss: 1.3071 - accuracy: 0.4122 - val_loss: 1.3005 - val_accuracy: 0.4164
Epoch 7/15
186/186 [==============================] - 1s 7ms/step - loss: 1.3070 - accuracy: 0.4122 - val_loss: 1.3006 - val_accuracy: 0.4164
Epoch 8/15
186/18